In [1]:
import xarray as xr
from datetime import datetime, timedelta
from pathlib import Path
import os
import pandas as pd

In [2]:
data_folder = Path("./data")

In [3]:
def read_year_netcdf(year):
    files = sorted(os.listdir(data_folder / str(year)))

    cams_year = []

    for path in files:
        date = path.split(".")[0].split("_")[-1]
        date = datetime.strptime(date, "%Y-%m-%d")

        cams_date = xr.open_dataset(data_folder / str(year) / path)    
        cams_date = cams_date.assign_coords(date=[date] * cams_date.time.shape[0])
        cams_date = cams_date.assign_coords(forecast_date=cams_date.date + cams_date.time)
        # cams_date['date'] = [date] * cams_date.time.shape[0]
        # cams_date['forecast_date'] = cams_date.date + cams_date.time

        cams_year.append(cams_date)
        
    return cams_year


In [17]:
def netcdf_to_df(cams_year, lat, lon):
    cams_year = cams_year.copy()
    level = 0

    df_year = []
    for cams_day in cams_year:
        df = cams_day.sel(
            latitude=lat, 
            longitude=lon,
            level=level,
            method="nearest"
        )

        df_year.append(df.to_dataframe().reset_index())

    df_year = pd.concat(df_year)
    df_year = df_year[df_year.time.dt.days == 3]
    
    return df_year


In [59]:
locations_stations = {
    "Parco S. Chiara": "Trento",
    "Via Bolzano": "Trento",
    "A22 (Avio)": "Avio",
    "Borgo Valsugana": "Borgo",
    "Riva del Gard": "Riva del Garda",
    "Monte Gaza": "Monte Gaza",
    "Rovereto": "Rovereto",
    "Piana Rotaliana": "Piana Rotaliana",
}

locations = {
    "Trento": (46.06787, 11.12108),
    "Borgo": (46.051716, 11.45419),
    "Riva del Garda": (45.89143, 10.844392),
    "Monte Gaza": (46.082529, 10.95804),
    "Rovereto": (45.892346, 11.039626),
    "Piana Rotaliana": (46.196812, 11.113394),
    "Avio": (45.742160, 10.970466),
}

years = [2020, 2021, 2022]

location_dfs = {}

for year in years:
    print(year)
    location_xr = read_year_netcdf(year)        
    location_df = []
    
    for station, location in locations_stations.items():
        print(station)
        
        lat, lon = locations[location]
        
        loc_year_df = netcdf_to_df(location_xr, lat, lon)
        loc_year_df["location"] = location
        loc_year_df["year"] = year
        loc_year_df["Stazione"] = station
        
        location_df.append(loc_year_df)
        
    location_dfs[(year, station)] = pd.concat(location_df)

dfs = pd.concat(location_dfs.values())


2020
Parco S. Chiara
Via Bolzano
A22 (Avio)
Borgo Valsugana
Riva del Gard
Monte Gaza
Rovereto
Piana Rotaliana
2021
Parco S. Chiara
Via Bolzano
A22 (Avio)
Borgo Valsugana
Riva del Gard
Monte Gaza
Rovereto
Piana Rotaliana
2022
Parco S. Chiara
Via Bolzano
A22 (Avio)
Borgo Valsugana
Riva del Gard
Monte Gaza
Rovereto
Piana Rotaliana


In [63]:
daily_dfs = dfs.groupby(["Stazione", dfs.forecast_date.dt.date]).mean().reset_index()

In [79]:
daily_dfs_subset = daily_dfs[['Stazione', 'forecast_date', 'no2_conc', 'pm10_conc', 'year']]
daily_dfs_subset = daily_dfs_subset.rename({
    "forecast_date": "Data",
    "year": "Year",
}, axis=1)

pm10 = daily_dfs_subset[
    ["Stazione", "Data", "pm10_conc", "Year"]
]

no2 = daily_dfs_subset[
    ["Stazione", "Data", "no2_conc", "Year"]
]

pm10["Inquinante"] = "PM10"
pm10 = pm10.rename({"pm10_conc": "Valore"}, axis=1)
no2["Inquinante"] = "Biossido di Azoto"
no2 = no2.rename({"no2_conc": "Valore"}, axis=1)

df_quasi_final = pd.concat([pm10, no2])
df_quasi_final["Year"] = df_quasi_final.Year.map(int)
df_quasi_final.to_csv("./cams_data_4d_daily_pm10_no2.csv", index=None)

In [75]:
pm10.columns, no2.columns

(Index(['Stazione', 'Data', 'Valore', 'Year', 'Inquinante'], dtype='object'),
 Index(['Stazione', 'Data', 'Valore', 'Valore', 'Inquinante'], dtype='object'))